Install requirements.

In [1]:
!pip install -r requirements.txt -qq

Download a wandb model.

In [1]:
import wandb

# define wandb environment
%env WANDB_LOG_MODEL=true
%env WANDB_API_KEY=53c099408fab02d1e4fff7386e8dfc1e759689a1

env: WANDB_LOG_MODEL=true
env: WANDB_API_KEY=53c099408fab02d1e4fff7386e8dfc1e759689a1


In [2]:
%%writefile wolof-translate/wolof_translate/utils/download_model.py
import shutil
import wandb
import glob
import os

def transfer_model(artifact_dir: str, model_name: str):
    """Transfer a download artifact into another directory
    
    Args:
        artifact_dir (str): The directory of the artifact
        model_name (str): The name of the model
    """
    # transfer the model inside the artifact to data/checkpoints/name_of_model
    os.makedirs(model_name, exist_ok=True)
    for file in glob.glob(f'{artifact_dir}/*'):
        shutil.copy(file, model_name)

    # delete the artifact
    shutil.rmtree(artifact_dir)

def download_artifact(artifact_name: str, model_name: str, type_: str = 'dataset'):
    """This function download an artifact from weights and bias and store it into a directory
    
    Args:
        artifact_name (str): name of the artifact
        model_name (str): name of the model
        type (str): type of the artifact. Default to 'directory'.
    """
    # download wandb model
    run = wandb.init()
    artifact = run.use_artifact(artifact_name, type=type_)
    artifact_dir = artifact.download()
    
    # transfer the artifact into another directory
    transfer_model(artifact_dir, model_name)
    
    # finish wandb
    wandb.finish()

Overwriting wolof-translate/wolof_translate/utils/download_model.py


In [3]:
from wolof_translate.utils.download_model import download_artifact

In [4]:
# download and transfer an artifact from weights and bias
artifact_name = 'oumar-kane-team/custom_transformer-fw-translation-bayes-hpsearch-v6/model_zokgpai:v0'
model_name = 'custom_transformer_v6_fw_best'

download_artifact(artifact_name, model_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: oumar-kane (oumar-kane-team). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model_zokgpai:v0, 207.83MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:9.0


In [5]:
import os
os.listdir(model_name)

['checkpoints.pth', 'checkpoints.json']

Rename the checkpoints if necessary.

In [6]:
# os.rename(f'{model_name}/checkpoints.json', f'{model_name}/best_checkpoints.json')
# os.rename(f'{model_name}/checkpoints.pth', f'{model_name}/best_checkpoints.pth')

-----------------

Send a directory as a wandb artifact.

In [12]:
%%writefile wolof-translate/wolof_translate/utils/send_model.py
import wandb

def add_directory(directory: str, artifact_name: str, project: str = 'fw_artifacts', entity: str = 'oumar-kane-team'):
    """Initialize a project and add checkpoints as artifact to wandb

    Args:
        directory (str): The directory where are stored the checkpoints
        artifact_name (_type_): The name of the artifact
        project (str, optional): The project name. Defaults to 'fw_artifacts'.
        entity (str, optional): The entity name. Defaults to 'oumar-kane-team'.
    """
    
    run = wandb.init(project=project, entity=entity)
    
    # add a directory as artifact to wandb
    artifact = wandb.Artifact(artifact_name, type='dataset')
    artifact.add_dir(directory)
    run.log_artifact(artifact)
    
    wandb.finish()

Overwriting wolof-translate/wolof_translate/utils/send_model.py


In [ ]:
from wolof_translate.utils.send_model import add_directory

In [ ]:
artifact_name = 'custom_transformer_v6_fw'
add_directory(artifact_name, artifact_name)

Compress the logs into an archive.

In [9]:
import shutil

model_type = 'transformer_custom_train_fw'

logs_path = f'data/logs/{model_type}'

file = 'version_6' # indicate the version name

shutil.make_archive('logs', 'zip', logs_path, file)

'/home/ec2-user/SageMaker/en_us/Wolof_traduction/logs.zip'